## Goede voornemens het hele jaar door
Python cloud functions en cloud storage objecten voor opslaan en ophalen van goede voornemens, plannen voor realisatie en reflectie daarop. 

In [1]:
import json
import requests

cloudfunction = 'https://8lgmayxgl6.execute-api.eu-central-1.amazonaws.com/default/wwdw'

req = {
  "body": {
    # "wwdwsession": "20220106g03704cj",
    # "documenthash": "uwwdw12345yxz",
    "loginname": "joeppyyyrrr"
    # "usersmake": True
   }
}

vdata = requests.post(cloudfunction, data=json.dumps(req))

vdata.json()


{'username': '',
 'wwdwid': 'wwdwid202201062ei5',
 'voornemen': [{'text': '',
   'step': [{'text': ''}],
   'thought': [{'text': ''}]}],
 'wwdwsession': '2022010786ea2fj1'}